# Exercises Meta Heuristics

In [75]:
import math  # type: ignore                                                           # Mathematical functions
import pandas as pd  # type: ignore                                                   # Data manipulation
import numpy as np  # type: ignore                                                    # Scientific computing
import matplotlib.pyplot as plt  # type: ignore                                       # Data visualization
from scipy.stats import binom as binomial  # type: ignore                             # Binomial distribution
from scipy.stats import norm as normal  # type: ignore                                # Normal distribution
from scipy.stats import poisson as poisson  # type: ignore                            # Poisson distribution
from scipy.stats import t as student  # type: ignore                                  # Student distribution
from scipy.stats import chi2  # type: ignore                                          # Chi-squared distribution
from scipy.stats import ttest_1samp  # type: ignore                                   # One-sample t-test
from scipy.stats import chisquare  # type: ignore                                     # Chi-squared test
from scipy.special import comb  # type: ignore                                        # Combinations
from mlxtend.frequent_patterns import apriori  # type: ignore                         # Apriori algorithm
from mlxtend.frequent_patterns import fpgrowth  # type: ignore                        # FP-growth algorithm
from mlxtend.frequent_patterns import association_rules  # type: ignore               # Association rules
from mlxtend.preprocessing import TransactionEncoder  # type: ignore                  # Transaction encoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  # type: ignore  # Discriminant Analysis
from tensorflow import keras  # type: ignore                                          # Deep Learning library
from tensorflow.keras import Model  # type: ignore                                    # Model class
from tensorflow.keras.layers import Input, Dense, BatchNormalization  # type: ignore  # Layers
from tensorflow.keras.utils import to_categorical  # type: ignore                     # One-hot encoding
from tensorflow.keras.optimizers import Adam  # type: ignore                         # Optimizer
from livelossplot import PlotLossesKeras  # type: ignore                              # Live plot
from keras.src.optimizers import RMSprop  # type: ignore                              # Optimizer
from sklearn.model_selection import train_test_split  # type: ignore                  # Train-test split
from simanneal import Annealer  # type: ignore                                        # Simulated Annealing
from inspyred import ec  # type: ignore                                                # Evolutionary Computation
import warnings  # type: ignore                                                       # Disable warnings
from Resources.Functions import *  # type: ignore                                     # Custom functions
warnings.filterwarnings("ignore")

# Guidelines
- To solve the problems below, always use simulated annealing and genetic algorithms. So you always provide two solutions. If you have the Tabu search Once implemented, you can even develop a third solution.
- You always have to write a cost function (objective function) yourself that is focused on the problem. You can reuse the cost function for the Simulated Annealing for the genetic algorithms. Note how the objective function should be optimized. You can - if necessary - add a minus sign and/or use rounding.
- Don't forget to also write a function that identifies other solution(s) somewhere in the solution space (e.g. random) or near the current solution(s).
- Experiment with some parameters such as the chance of crossover, or mutation in genetics algorithms.
- In some assignments it is best to also provide lower and upper limits (lower and upper).

## Theoretical questions

### Question 0: Simulated Annealing
The simulated annealing algorithm is a probabilistic optimization algorithm that is used to find the global optimum of a function. The algorithm is inspired by the annealing process in metallurgy, where a material is heated and then slowly cooled to increase the size of its crystals. The algorithm works by starting at a random point in the search space and then iteratively moving to a new point in the search space. The algorithm accepts moves to new points that improve the objective function value, but also accepts moves to points that decrease the objective function value with a certain probability. This probability decreases over time, allowing the algorithm to escape local optima and find the global optimum.

$$ P(\text{accept worse}) = e^{-\frac{\Delta E}{T}} $$

In [34]:
class RastriginProblem(Annealer):
    def energy(self):
        x1 = self.state[0]
        x2 = self.state[1]
        sum = (10 * 2 + x1**2 + x2**2)
        sum = sum - 10 * math.cos(2 * math.pi * x1) - 10 * math.cos(2 * math.pi * x2)
        return sum # Maximize = return -sum

    def move(self):
        i = np.random.randint(0,2) # Randomly select a dimension (between 0 and 1)
        self.state[i] += np.random.normal(0, 0.1)
        self.state[i] = np.clip(self.state[i], -5.12, 5.12)

initial_state =  np.random.uniform(-5.12,5.12, size=2)
rastrigin = RastriginProblem(initial_state)

rastrigin.Tmax = 25000.0                # Max (starting) temperature
rastrigin.Tmin = 2.5                    # Min (ending) temperature
rastrigin.updates = 10                  # Number of updates (On the screen)
rastrigin.steps = 100000                # Number of iterations
route, distance = rastrigin.anneal()    # Start the annealing

print(f"Route: {route}\nDistance: {distance}")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
    2.50000          3.45    72.63%    35.94%     0:00:01     0:00:00

Route: [0.00437066 0.00289638]
Distance: 0.005453844800690888


### Question 0: Traveling Salesman
The traveling salesman problem (TSP) is a classic optimization problem where the goal is to determine the shortest tour of a collection of `n` "cities" (i.e. nodes), visiting each city exactly once and returning to the starting city. The distance between each pair of cities is known, and the goal is to find the shortest possible tour that visits each city exactly once and returns to the starting city.

$$ \text{Minimize} \sum_{i=1}^{n} \sum_{j=1}^{n} c_{ij} \cdot x_{ij} $$


![Traveling Salesman](Images/travelingSalesman.png)

In [3]:
# Matrix where the distances between the cities are stored (The matrix is symmetric)
# 2 dimensions (5*5) (x,y)
distance_matrix2 = np.array([[0, 100, 125, 100,  75],
                            [100, 0,  50,  75, 100],
                            [125, 50,  0, 100, 125],
                            [100, 75, 100,   0, 50],
                            [75, 100, 125,  50,  0]])

In [35]:
class TravellingSalesmanProblem(Annealer):
    def energy(self): # Calculates the length of the route.
        dist = 0
        for i in range(len(self.state)):
            dist += distance_matrix2[self.state[i - 1], self.state[i]]
        return dist

    def move(self): # Swaps two cities in the route.
        a = np.random.randint(0, len(self.state) - 1)
        b = np.random.randint(0, len(self.state) - 1)
        self.state[a], self.state[b] = self.state[b], self.state[a]

initial_state = [0, 4, 1, 3, 2] # Random initial route 0 -> 4 -> 1 -> 3 -> 2 (Index of the cities)
route, distance = TravellingSalesmanProblem(initial_state).anneal()

print(f"Route: {route}\nDistance: {distance}")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


Route: [0, 4, 3, 1, 2]
Distance: 375


### Question 1: The Backpack
You will find yourself in a **secret room** equipped with a door with a **time lock**. You'll see a timer countdown telling you that you only have five minutes left before the door will be locked forever. In front of you are **valuable objects**, each with their own **yield and weight**. You have a backpack that can carry an absolute maximum weight of `750 grams`. Put together the optimal backpack. You should end up with an optimal yield of `1458` (or at least a value close to that).

$$ \text{Maximize} \sum_{i=1}^{n} v_i \cdot x_i $$

In [20]:
knapsackItems = pd.read_csv('../Data/KnapsackItems.csv', sep=',')
display(knapsackItems.head())

,Voorwerpen,Gewichten(gr),Waarde
0,Voorwerp 1,70,135
1,Voorwerp 2,73,139
2,Voorwerp 3,77,149
3,Voorwerp 4,80,150
4,Voorwerp 5,82,156


In [39]:
class TheBackpackProblem(Annealer):
    def energy(self):
        solution = self.state
        total_weight = (solution * weight_items).sum()
        if total_weight > 750:
            total_value = 0
        else:
            total_value = (solution * value_items).sum()
        return -total_value
    
    def move(self):
        i = np.random.randint(0, len(self.state))
        self.state[i] = not self.state[i] # not = -1

weight_items = knapsackItems['Gewichten(gr)']
value_items = knapsackItems['Waarde']
initial_state = np.random.choice([0,1], size=len(weight_items)) # Size = all rows of the dataframe (CSV)
route, distance = TheBackpackProblem(initial_state).anneal()

print(f"Route: {route}\nDistance: {-distance}")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


Route: [1 0 1 0 1 0 1 1 1 0 0 0 0 1 1]
Distance: 1458


### Question 2: The Gutter
You are responsible for the design of gutters where the production costs must be as low as possible. It is therefore necessary that the gutters have an optimal cross-section with the available material so that leaves and dirt can be easily removed. The company you work for purchases **metal plates** that have a width of `1m. In other words H + W + H` see drawing-must be less than or equal to `1m`. Determine the **ideal `width B` and `height H`** of the gutter that you can make from the `1m` plates.

$$ \text{Maximize} \frac{1}{2} \cdot B \cdot H $$

![Gutter](Images/theGutter.png)

In [42]:
class TheGutterProblem(Annealer):
    def energy(self):
        b = self.state[0]
        h = (1 - b) / 2
        return -b * h

    def move(self):
        b = self.state[0]
        b += np.random.normal(0, 0.1)
        self.state[0] = np.clip(b, 0, 1)

initial_state = np.random.uniform(0,1, size=1)
route, distance = TheGutterProblem(initial_state).anneal()

print(f"Route: {route}\nDistance: {-distance}")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
    2.50000         -0.09    99.80%    46.80%     0:00:01     0:00:00

Route: [0.50000293]
Distance: 0.1249999999957189


### Question 3: The Football Stadium
The local sports club wants to build a new stadium. The perimeter of the sports field should be `400m`, and at the same time we want to ensure that the central midfield has a maximum surface area. Determine the ideal length and width ratio.

$$ \text{Maximize } L \cdot B $$

![The Football Stadium](Images/theFootballStadium.png)

In [73]:
class TheFootballStadium(Annealer):
    def energy(self):
        l = self.state[0]
        b = 400 - 2 * l
        area = l * b
        return -area

    def move(self):
        l = self.state[0]
        l += np.random.normal(0, 0.1)
        self.state[0] = np.clip(l, 0, 200)

initial_state = np.random.uniform(0, 200, size=1)
route, distance = TheFootballStadium(initial_state).anneal()

print(f"Route: {route}\nDistance: {-distance}")

L = route[0]
B = 200 - L
max_area = L * B

print(f"\nOptimal Width (L): {L:.2f} m")
print(f"Optimal Height (B): {B:.2f} m")
print(f"Maximum Area: {max_area:.2f} m^2")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


Route: [100.00005621]
Distance: 19999.99999999368

Optimal Width (L): 100.00 m
Optimal Height (B): 100.00 m
Maximum Area: 10000.00 m^2


### Question 4: Optimization problem

Given the following objective function to be maximized:


$ obj = 0.2 + x_1^2 + x_2^2 - 0.1 \cos(6\pi x_1) - 0.1 \cos(6\pi x_2) $


With the following restrictions: $-1.0 \leq x_i \leq 1.0$ met $i=1,2$

Find a good solution.

In [41]:
class OptimalisatieProblem(Annealer):
    def energy(self):
        x = self.state[0]
        y = self.state[1]
        return -(0.2 + x**2 + y**2 - 0.1 * math.cos(6 * math.pi * x) - 0.1 * math.cos(6 * math.pi * y))

    def move(self):
        i = np.random.randint(0,2)
        self.state[i] += np.random.normal(0, 0.1)
        self.state[i] = np.clip(self.state[i], -1, 1)

initial_state = np.random.uniform(-1,1, size=2)
route, distance = OptimalisatieProblem(initial_state).anneal()

print(f"Route: {route}\nDistance: {-distance}")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


Route: [-1. -1.]
Distance: 2.0


### Question 5: Quiz

1. Hebben bovengemiddelde chromosomen altijd een beter nageslacht? 
2. Kan je met cross-over de hele zoekruimte verkennen? Zijn er beperkingen van deze verkenning?
3. Waarom speelt de kans op mutatie een belangrijke rol?
4. Waarom start je bij Simulated Annealing het best met een gerandomiseerde vector?
5. Waarom moet de temperatuur bij simulated annealing afnemen?

### General explanation:

What is what?
- The output `Route` is the solution of the problem.
- The output `Distance` is the value of the objective function.
- The `energy` method is the objective function.
- The `move` method is the function that identifies other solution(s) somewhere in the solution space.
- The `initial_state` is the random initial solution.
- The `anneal` method is the function that starts the annealing process.
- The `objective function` is the function that we want to maximize or minimize so `energy` method is the objective function.